# MODEL_SVM

**Let's start by uploading the dataset already normalized and defining the X and y for our model**

In [43]:
import pandas as pd
import numpy as np

In [44]:
churn_svm = pd.read_csv('Churn_Norm.csv')

In [45]:
churn_svm.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Int,Geography_Germany,Geography_Spain,Balance_Int
0,0.538,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735,1.0,1.0,0.0,0.0,0.0
1,0.516,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709,0.0,1.0,0.0,1.0,1.0
2,0.304,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654,1.0,1.0,0.0,0.0,1.0
3,0.698,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120,0.0,1.0,0.0,0.0,0.0
4,1.000,0.337838,0.2,0.500246,0.000000,1.0,1.0,0.395400,0.0,1.0,0.0,1.0,1.0


In [46]:
X = churn_svm.drop(['Exited'], axis=1)
y = churn_svm.Exited

In [47]:
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Int,Geography_Germany,Geography_Spain,Balance_Int
0,0.538,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735,1.0,0.0,0.0,0.0
1,0.516,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709,1.0,0.0,1.0,1.0
2,0.304,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654,1.0,0.0,0.0,1.0
3,0.698,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120,1.0,0.0,0.0,0.0
4,1.000,0.337838,0.2,0.500246,0.000000,1.0,1.0,0.395400,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.842,0.283784,0.5,0.000000,0.333333,1.0,0.0,0.481341,0.0,0.0,0.0,0.0
9996,0.332,0.229730,1.0,0.228657,0.000000,1.0,1.0,0.508490,0.0,0.0,0.0,1.0
9997,0.718,0.243243,0.7,0.000000,0.000000,0.0,1.0,0.210390,1.0,0.0,0.0,0.0
9998,0.844,0.324324,0.3,0.299226,0.333333,1.0,0.0,0.464429,0.0,1.0,0.0,1.0


In [48]:
y

0       1.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
9995    0.0
9996    0.0
9997    1.0
9998    1.0
9999    0.0
Name: Exited, Length: 10000, dtype: float64

**We will try our first different models:**

**1. LINEAR**

**2. RBF**

**3. POLY**

## SMV (Linear)

In [49]:
from sklearn.svm import SVC

In [50]:
svm_linear = SVC(kernel='linear', C=10)

In [51]:
svm_linear.fit(X,y)

SVC(C=10, kernel='linear')

In [52]:
print('Linear SVC will give us:', round(svm_linear.score(X,y),3))

Linear SVC will give us: 0.796


**We split the data into train_test**

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)

In [57]:
svm_linear.fit(X_train, y_train)

svm_linear.fit(X_test, y_test)

print('Linear SVC trained will give us:', round(svm_linear.score(X_train,y_train),3))

print('Linear SVC test will give us:', round(svm_linear.score(X_test,y_test),3))

Linear SVC trained will give us: 0.795
Linear SVC test will give us: 0.809


**Conclusions**

1. Our SVC trained will give the exact same results.
2. Our SVC test will give us better results, which is overfitting.

## SMV_RBF

In [58]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
X = StandardScaler().fit_transform(X)

In [62]:
svm_rbf = SVC(kernel='rbf', C=10)

In [63]:
svm_rbf.fit(X,y)

SVC(C=10)

In [64]:
print('RBF SVC will give us:', round(svm_rbf.score(X,y),2))

RBF SVC will give us: 0.89


In [72]:
svm_rbf.fit(X_train, y_train)

svm_rbf.fit(X_test, y_test)

print('RBF SVC trained will give us:', round(svm_rbf.score(X_train,y_train),3))

print('RBF SVC test will give us:', round(svm_rbf.score(X_test,y_test),3))

RBF SVC trained will give us: 0.826
RBF SVC test will give us: 0.894


**Conclusions**

1. Our RBF trained will give a lower accuracy but higher than the initial 80%.
2. Our RBF test will give us better results, which is overfitting.

## SMV_POLY

In [68]:
svm_poly = SVC(kernel='poly', C=10)

svm_poly.fit(X,y)

In [71]:
print('Poly SVC will give us:', round(svm_poly.score(X,y),3))

Poly SVC will give us: 0.87


In [74]:
svm_poly.fit(X_train, y_train)

svm_poly.fit(X_test, y_test)

print('Poly SVC trained will give us:', round(svm_poly.score(X_train,y_train),3))

print('Poly SVC test will give us:', round(svm_poly.score(X_test,y_test),3))

Poly SVC trained will give us: 0.826
Poly SVC test will give us: 0.886


**Conclusions**

1. Our Poly SVC trained will give a lower accuracy but higher than the initial 80%.
2. Our Poly SVC test will give us better results, which is overfitting.

**To sum up:**

The best model is RBF because it will give us an 0,89% accuracy.
Followed by Poly with an 0,87% and last Linear with an 0,79%.

## PCA

Let's see if our results improve with a PCA

In [19]:
from sklearn.decomposition import PCA

In [20]:
pca = PCA(n_components=8)

In [21]:
pc = pca.fit_transform(X, y)

In [22]:
pc.shape

(10000, 8)

In [77]:
svm_linear.fit(pc, y)
print('Linear SVC with PCA will give us:', round(svm_linear.score(pc,y),3))

Linear SVC with PCA will give us: 0.796


In [78]:
svm_rbf.fit(pc, y)
print('Linear SVC with PCA will give us:', round(svm_rbf.score(pc,y),3))

Linear SVC with PCA will give us: 0.856


In [79]:
svm_poly.fit(pc, y)
print('Linear SVC with PCA will give us:', round(svm_poly.score(pc,y),3))

Linear SVC with PCA will give us: 0.822


**With a PCA of 8 components, the results will be worse**

In [80]:
pca2 = PCA(n_components=2)

In [81]:
pc2 = pca2.fit_transform(X, y)

In [82]:
pc2.shape

(10000, 2)

In [83]:
svm_linear.fit(pc2, y)
print('Linear SVC with PCA will give us:', round(svm_linear.score(pc2,y),3))

Linear SVC with PCA will give us: 0.796


In [84]:
svm_rbf.fit(pc2, y)
print('Linear SVC with PCA will give us:', round(svm_rbf.score(pc2,y),3))

Linear SVC with PCA will give us: 0.811


In [85]:
svm_poly.fit(pc2, y)
print('Linear SVC with PCA will give us:', round(svm_poly.score(pc2,y),3))

Linear SVC with PCA will give us: 0.796


**With a PCA of 2 components, the results will be worse than with 8**

## LDA

Let's see if our results improve with a LDA

In [26]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [88]:
lda = LinearDiscriminantAnalysis(n_components=1)

lda = lda.fit(X,y)

lda.score(X,y)

0.8075

**Our results are not improving with LDA**

## Conclusion

**Our model selected would be RBF**